In [1]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 67.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 7.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM ,BitsAndBytesConfig

import time
import json
import os

In [3]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: read).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in c

In [4]:
model_id="meta-llama/Llama-2-7b-chat-hf"

In [5]:
quantization_config = BitsAndBytesConfig(load_in_4bit=True)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quantization_config,
    device_map='auto',
)

tokenizer = AutoTokenizer.from_pretrained(model_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [7]:
def get_llama2_response(prompt, max_new_tokens=10):
    start_time = time.time()
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(**inputs, max_new_tokens=max_new_tokens, temperature=0.00001)
    end_time = time.time()
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    elapsed_time = end_time - start_time
    print(f"Time taken: {elapsed_time} seconds")
    return response, elapsed_time

# File paths
#output_json_path = '/IAP_data/iap-vl/IAP/QA/experiment/llama2_results.json'

# Sample prompt
prompt = '''
[INST]
can you give me python code for extract capital latters from a string
[INST]
'''

# Get the current response
output, elapsed_time = get_llama2_response(prompt, max_new_tokens=1000)
print(output)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1830: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


Time taken: 26.05693554878235 seconds

[INST]
can you give me python code for extract capital latters from a string
[INST]

Sure! Here is an example of how you can extract capital letters from a string in Python:
```
def extract_capital_letters(str):
    # Use a regular expression to match capital letters
    regex = r'[A-Z]'
    # Use the replace() method to replace the capital letters with an empty string
    return str.replace(regex, '')

# Test the function
print(extract_capital_letters('Hello World')) # Output: hW
```
In this example, the `extract_capital_letters()` function uses a regular expression to match any character that is a capital letter (i.e. any character that is uppercase). The `replace()` method is then used to replace any capital letters with an empty string.

You can also use the `title()` function to extract capital letters from a string. Here is an example:
```
def extract_capital_letters(str):
    # Use the title() function to convert the string to title case
  

In [10]:
prompt = '''
[INST]
Classify the text into neutral, negative or positive.
Text: I just love it
[/INST]
'''
# Get the current response
output, elapsed_time = get_llama2_response(prompt, max_new_tokens=1000)
print(output)

Time taken: 5.07925271987915 seconds

[INST]
Classify the text into neutral, negative or positive. 
Text: I just love it
[/INST]
Based on the text you provided, I would classify it as positive. The use of the word "love" and the enthusiastic tone suggest that the speaker is expressing a positive emotion.


In [13]:
prompt = '''
[INST]
what is the hindi translation of the text
Text: I like eating
[/INST]
'''

# Get the current response
output, elapsed_time = get_llama2_response(prompt, max_new_tokens=1000)
print(output)

Time taken: 5.0920233726501465 seconds

[INST]
what is the hindi translation of the text 
Text: I like eating
[/INST]
The Hindi translation of the text "I like eating" is:

मैं खाना पसंद हूँ (Maine khana pasand hoon)


### Few Shot Prompting

prompt = '''
[INST] To do a "farduddle" means to jump up and down really fast. An example of a sentence that uses
the word farduddle is:  [/INST]
'''

# Get the current response
output, elapsed_time = get_llama2_response(prompt, max_new_tokens=1000)
print(output)

In [15]:
prompt = '''
[INST] A "whatpu" is a small, furry animal native to Tanzania. An example of a sentence that uses
the word whatpu is: [/INST]
We were traveling in Africa and we saw these very cute whatpus.
[INST] To do a "farduddle" means to jump up and down really fast. An example of a sentence that uses
the word farduddle is: [/INST]
'''
# Get the current response
output, elapsed_time = get_llama2_response(prompt, max_new_tokens=1000)
print(output)

Time taken: 2.1514437198638916 seconds

[INST] A "whatpu" is a small, furry animal native to Tanzania. An example of a sentence that uses
the word whatpu is: [/INST]
We were traveling in Africa and we saw these very cute whatpus.
[INST] To do a "farduddle" means to jump up and down really fast. An example of a sentence that uses
the word farduddle is: [/INST]
The children were having a great time farduddling in the playground.


### Few Shot Prompting for Strcuture

In [16]:
prompt = '''
[INST] <<SYS>>
Classify the text into neutral, negative or positive.
<</SYS>>

I like this.[/INST]
positive
[INST] I hate this [/INST]
negative
[INST] this is ok [/INST]
'''
# Get the current response
output, elapsed_time = get_llama2_response(prompt, max_new_tokens=1000)
print(output)

Time taken: 1.0052988529205322 seconds

[INST] <<SYS>>
Classify the text into neutral, negative or positive.
<</SYS>>

I like this.[/INST]
positive
[INST] I hate this [/INST]
negative
[INST] this is ok [/INST]
neutral


In [ ]:
Chain of Thought Prompting

In [17]:
prompt = '''
[INST] True or False. The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82. [/INST]
'''

# Get the current response
output, elapsed_time = get_llama2_response(prompt, max_new_tokens=1000)
print(output)

Time taken: 5.647612810134888 seconds

[INST] True or False. The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82. [/INST]
True. The sum of the odd numbers in the group is 15 + 32 + 5 = 52, which is an even number.


In [18]:
prompt = '''
[INST] The odd numbers in this group add up to an even number: 4, 8, 9, 15, 12, 2, 1. [/INST]
A: Odd numbers in the group are: 9, 15, 1. Sum of them is 25. 25 is odd. The answer is False.
[INST] The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82. [/INST]
'''
# Get the current response
output, elapsed_time = get_llama2_response(prompt, max_new_tokens=1000)
print(output)

Time taken: 4.617768287658691 seconds

[INST] The odd numbers in this group add up to an even number: 4, 8, 9, 15, 12, 2, 1. [/INST]
A: Odd numbers in the group are: 9, 15, 1. Sum of them is 25. 25 is odd. The answer is False.
[INST] The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82. [/INST]
A: Odd numbers in the group are: 15, 5, 13. Sum of them is 33. 33 is odd. The answer is False.


### Prompting for Chat

In [19]:
prompt = '''
[INST] <<SYS>>
Respond only by quoting from the TV series Friends
<</SYS>>

Hi[/INST]
'''
# Get the current response
output, elapsed_time = get_llama2_response(prompt, max_new_tokens=1000)
print(output)

Time taken: 1.46189284324646 seconds

[INST] <<SYS>>
Respond only by quoting from the TV series Friends
<</SYS>>

Hi[/INST]
"How you doin'?"


In [20]:
prompt = '''
[INST] <<SYS>>
Respond only by quoting from the TV series Friends
<</SYS>>

Hi[/INST]
"How you doin'?"
[INST] whats the capital of India?[/INST]
'''
# Get the current response
output, elapsed_time = get_llama2_response(prompt, max_new_tokens=1000)
print(output)

Time taken: 6.782391309738159 seconds

[INST] <<SYS>>
Respond only by quoting from the TV series Friends
<</SYS>>

Hi[/INST]
"How you doin'?"
[INST] whats the capital of India?[/INST]
"Oh, you want to know the capital of India? Well, let me see... *giggles* Oh, right! It's... (exaggerated whisper) Delhi! *wink*"


In [21]:
prompt = '''
[INST] <<SYS>>
Respond only by using emojis
<</SYS>>

Hi[/INST]
👋
[INST] whats the capital of India?[/INST]
🇮🇳
[INST] show me some dance move [/INST]
'''
# Get the current response
output, elapsed_time = get_llama2_response(prompt, max_new_tokens=1000)
print(output)

Time taken: 2.2527148723602295 seconds

[INST] <<SYS>>
Respond only by using emojis
<</SYS>>

Hi[/INST]
👋
[INST] whats the capital of India?[/INST]
🇮🇳
[INST] show me some dance move [/INST]
💃🕺


In [22]:
prompt = '''
[INST]
Hi, my name is Yash and I am 27[/INST]
Hello Yash
[INST] whats the capital of India?[/INST]
Delhi
[INST] whats my name? [/INST]
Your name is Yash.
[INST] what is my age?[/INST]
'''
# Get the current response
output, elapsed_time = get_llama2_response(prompt, max_new_tokens=1000)
print(output)

Time taken: 2.3619446754455566 seconds

[INST] 
Hi, my name is Yash and I am 27[/INST]
Hello Yash
[INST] whats the capital of India?[/INST]
Delhi
[INST] whats my name? [/INST]
Your name is Yash.
[INST] what is my age?[/INST]
You are 27 years old.
